# Procesamiento básico de nube de puntos usando Open3D

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/oscar-ramos/robotica-autonoma-python/blob/main/5-Nubes-Puntos/5-3-Procesamiento-basico-open3d.ipynb)

In [ ]:
!pip install -q open3d

In [ ]:
import numpy as np

import open3d as o3d
import plotly.graph_objs as go

Se utilizará una nube de puntos, con colores, de una mesa con objetos sobre la mesa. La nube de puntos en formato PCD se encuentra originalmente [aquí](https://github.com/udacity/RoboND-Perception-Exercises/tree/master/Exercise-1), y por facilidad se ha copiado a este repositorio.

In [ ]:
!wget -q https://raw.githubusercontent.com/oscar-ramos/robotica-autonoma-python/main/5-Nubes-Puntos/datos/tabletop.pcd

## 1.&nbsp;Lectura y Visualización de la Nube de Puntos

In [ ]:
# Leer el archivo .pcd file
nube = o3d.io.read_point_cloud("tabletop.pcd")

# Indicar si la nube contiene información del color
if nube.has_colors():
    print("La nube de puntos tiene información de colores")

In [ ]:
# Extracción de colores para cada punto
colores = np.asarray(nube.colors)

# Extracción de coordenadas de cada punto
puntos = np.asarray(nube.points)
x = puntos[:, 0]
y = puntos[:, 1]
z = puntos[:, 2]

# Mostrar información sobre los puntos
# puntos.shape
print("La nube tiene {} puntos".format(puntos.shape[0]))

In [ ]:
# Algunos puntos X, Y, Z
print("10 primeros X:", np.round(x[:10],3))
print("10 primeros Y:", np.round(y[:10],3))
print("10 primeros Z:", np.round(z[:10],3))

In [ ]:
def visualizar_nube(nube):
    # Recuperación de colores y coordenadas de la nube de puntos
    colores = np.asarray(nube.colors)
    puntos = np.asarray(nube.points)
    x = puntos[:, 0]; y = puntos[:, 1]; z = puntos[:, 2]
    # Creación de un gráfico de puntos 3D (scatter plot) de Plotly
    fig = go.Figure(data=[go.Scatter3d(x=x, y=y, z=z, mode='markers',
                                    marker=dict(size=2, color=colores, opacity=0.6)
                                    )])
    # Mitad del rango de los datos
    rango = 0.5*np.max([np.ptp(x), np.ptp(y), np.ptp(z)])
    # Igualar escalas en cada eje
    fig.update_layout(scene=dict(
        xaxis=dict(range=[np.mean(x)-rango, np.mean(x)+rango], title='X'),
        yaxis=dict(range=[np.mean(y)-rango, np.mean(y)+rango], title='Y'),
        zaxis=dict(range=[np.mean(z)-rango, np.mean(z)+rango], title='Z'),
        aspectratio=dict(x=1, y=1, z=1),
        aspectmode='manual'
    ))
    fig.show()

In [ ]:
visualizar_nube(nube)

## 2.&nbsp;Submuestreo

Se muestreará la nube de puntos utilizando el método `voxel_down_sample(voxel_size)` de la nube de puntos de Open3D, donde `voxel_size` es el tamaño del voxel. El algoritmo mantiene un único punto por cada voxel, por lo que un mayor tamaño de voxel generará una menor cantidad de puntos (un muestreo más extremo). Consecuentemente, un menor valor del tamaño del voxel mantendrá más puntos de la nube.

In [ ]:
# Muestreo
nube_muestreada = nube.voxel_down_sample(voxel_size=0.01)

In [ ]:
# Extracción de los nuevos puntos
puntos_muestreo = np.asarray(nube_muestreada.points)
# Información sobre la nueva nube
print("Número de puntos luego del muestreo:", puntos_muestreo.shape[0])

In [ ]:
visualizar_nube(nube_muestreada)

## 3.&nbsp;Filtro de región de interés

Para seleccionar una región de inteerés se utilizará el método `crop(bbox)` que corta la nube para quedarse solamente con la región de interés `bbox` ("bounding box"). Esta región de interés se puede obtener con el método `geometry.AxisAlignedBoundingBox` especificando adecuadamente los límites inferior y superior de X, Y, Z.

In [ ]:
# Crear una región de interés ("bounding box")
bbox = o3d.geometry.AxisAlignedBoundingBox(min_bound=(float('-inf'), float('-inf'), 0.5),
                                           max_bound=(float('inf'), -1.35, 2))

# Recortar la nube utilizando la región de interés (RoI)
nube_roi = nube.crop(bbox)

In [ ]:
# Visualizar el resultado
visualizar_nube(nube_roi)

## 4.&nbsp;RANSAC

In [ ]:
# Segmentación del plano
modelo_plano, inliers = nube_roi.segment_plane(distance_threshold = 0.02,
                                               ransac_n = 3,
                                               num_iterations = 1000)

In [ ]:
# Extracción de inliers y outliers
nube_inliers = nube_roi.select_by_index(inliers)
nube_outliers = nube_roi.select_by_index(inliers, invert=True)

In [ ]:
# Visualización de inliers
visualizar_nube(nube_inliers)

In [ ]:
# Visualización de outliers
visualizar_nube(nube_outliers)

## 5.&nbsp;Filtraje estadístico

Usualmente existe ruido debido a factores externos como el polvo en el ambiente, la humedad en el aire o la presencia de varias fuentes de luz. Este ruido genera valores atípicos dispersos que corrompen los resultados de la nube de puntos y complican la estimación de sus características.

Una forma de eliminar estos valores atípicos consiste en realizar un análisis estadístico en el vecindario de cada punto y eliminar aquellos puntos que no cumplan con ciertos criterios. En este caso se utilizará un "filtro de eliminación de valores atípicos estadísticos".

El funcionamiento de este filtro es como sigue. Para cada punto en la nube de puntos, este filtro calcula la distancia a todos sus vecinos y luego calcula una distancia media. Al asumir una distribución gaussiana, se consideran valores atípicos, y se eliminan de la nube de puntos, todos aquellos puntos cuyas distancias medias estén fuera de un intervalo definido por la media de las distancias globales más la desviación estándar. En Open3D esto se realiza con el método `remove_statistical_outlier`.

In [ ]:
# Filtraje estadístico de ruido
nube_filtrada, ind = nube_outliers.remove_statistical_outlier(nb_neighbors=50, std_ratio=1.0)

In [ ]:
# Visualización de la salida
visualizar_nube(nube_filtrada)

## 6.&nbsp;Clusterización

In [ ]:
from random import randint
def get_color_list(n_colores):
    """ Retorna una lista de colores aleatorios
    """
    lista_colores = []
    for i in range(n_colores):
        lista_colores.append([randint(0,255), randint(0,255), randint(0,255)])
    return lista_colores

Se realizará clusterización utilizando el algoritmo DBSCAN que es un método de la nube de open3D llamado `cluster_dbscan`.

In [ ]:
# Clusters usando DBSCAN
etiquetas = np.array(nube_filtrada.cluster_dbscan(eps = 0.08,
                                                  min_points = 100,
                                                  print_progress = False))

print("Clusters únicos usando DBSCAN:", np.unique(etiquetas))

In [ ]:
# Índices de cada cluster
indices_cluster = []
for id_cluster in range(np.max(etiquetas) + 1):
    indices_cluster.append(np.where(etiquetas == id_cluster)[0])

# Número de puntos en cada cluster
for idx, indices in enumerate(indices_cluster):
    print("En el cluster {} hay {} puntos.".format(idx, len(indices)))

In [ ]:
# Filtraje de clústeres muy pequeños o muy grandes
MIN_CLUSTER = 100
MAX_CLUSTER = 10000

indices_filtrados = []
for indices in indices_cluster:
    if len(indices) >= MIN_CLUSTER and len(indices) <= MAX_CLUSTER:
        indices_filtrados.append(indices)

In [ ]:
# Obtener una lista de colores para los clústeres
colores_clusters = get_color_list(len(indices_filtrados))

# Crear una nueva nube de puntos para almacenar los clústeres con color
nube_cluster = o3d.geometry.PointCloud()
puntos = []; colores = []

# Iterar los clústeres y asignar colores a los puntos
for cluster_idx, indices in enumerate(indices_filtrados):
    puntos_cluster = np.asarray(nube_filtrada.points)[indices]
    color_cluster = colores_clusters[cluster_idx]
    puntos.extend(puntos_cluster)
    colores.extend([color_cluster]*len(indices))

# Set the points and colors for the new point cloud
nube_cluster.points = o3d.utility.Vector3dVector(puntos)
nube_cluster.colors = o3d.utility.Vector3dVector(colores)

In [ ]:
visualizar_nube(nube_cluster)

**Extracción de un objeto usando los índices del clúster**

In [ ]:
# Número del objeto
N = 1

# Puntos y colores correspondientes al objeto
puntos = np.asarray(nube_filtrada.points)[indices_filtrados[N]]
colores = np.asarray(nube_filtrada.colors)[indices_filtrados[N]]

# Nube de puntos conteniendo solo el objeto
objeto = o3d.geometry.PointCloud()
objeto.points = o3d.utility.Vector3dVector(puntos)
objeto.colors = o3d.utility.Vector3dVector(colores)

In [ ]:
visualizar_nube(objeto)

## 7.&nbsp;Vector de características del objeto

Una vez que se tiene objetos separados, se puede caracterizar estos objetos utilizando diversas características. Por ejemplo, aquí se mostrará un histograma de color concatenado con un histograma de normales a la superficie.

Este vector de características puede ser luego utilizado como entrada a un clasificador, con el fin de entrenar un sistema para el reconocimiento del objeto.

In [ ]:
def calcular_histograma_color(nube):
    """Calcula un histograma de colores usando los puntos de la nube
    """
    colores = np.asarray(nube.colors)
    # Convertir colores de [0, 1] al rango [0, 255]
    colores = (colores*255).astype(int)
    # Listas que contendrán valores de color
    R_vals = []; G_vals = []; B_vals = []
    for color in colores:
        R_vals.append(color[0])
        G_vals.append(color[1])
        B_vals.append(color[2])
    # Calcular histogramas
    R_hist = np.histogram(R_vals, bins=32, range=(0, 256))
    G_hist = np.histogram(G_vals, bins=32, range=(0, 256))
    B_hist = np.histogram(B_vals, bins=32, range=(0, 256))
    # Concatenar y normalizar el histograma
    hist_features = np.concatenate((R_hist[0], G_hist[0], B_hist[0])).astype(np.float64)
    normed_features = hist_features / np.sum(hist_features)
    return normed_features

def calcular_histograma_normales(nube):
    """
    Calcula el histograma de normales para una nube de puntos
    """
    # Estimar las normales
    nube.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.03, max_nn=30))
    normales = np.asarray(nube.normals)
    # Separar los componentes normales
    norm_x = normales[:, 0]; norm_y = normales[:, 1]; norm_z = normales[:, 2]
    # Calcular histogramas de los valores normales
    norm_x_hist, _ = np.histogram(norm_x, bins=32, range=(-1, 1), density=True)
    norm_y_hist, _ = np.histogram(norm_y, bins=32, range=(-1, 1), density=True)
    norm_z_hist, _ = np.histogram(norm_z, bins=32, range=(-1, 1), density=True)
    # concatenar los histogramas
    hist_features = np.concatenate((norm_x_hist, norm_y_hist, norm_z_hist))
    return hist_features

In [ ]:
chists = calcular_histograma_color(objeto)
nhists = calcular_histograma_normales(objeto)

# Feature conteniendo histograma de color y de normales
feature = np.concatenate((chists, nhists))

print("Elementos del vector de características:", feature.shape[0])